In [36]:
import numpy as np
from dec.schur import schur, schurλ, bmat, rand
from dec.spectral import I_diag

In [37]:
n, m = 5, 2
B = np.arange(n*m).reshape((n,m))
C = np.arange(n*m).reshape((m, n))
D = np.arange(m*m).reshape((m,m))
print(B)
print(C)
print(D)
print(B.dot(np.eye(B.shape[-1], B.shape[-1])))
print(C.dot(np.eye(C.shape[-1], C.shape[-1])))

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
[[0 1 2 3 4]
 [5 6 7 8 9]]
[[0 1]
 [2 3]]
[[ 0.  1.]
 [ 2.  3.]
 [ 4.  5.]
 [ 6.  7.]
 [ 8.  9.]]
[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]]


In [38]:
def fourier_sin(a, b):
    r'''
    Corresponds to :math:`f(x) \mapsto \int_{x+a}^{x+b} f(\xi) \sin(\xi) d\xi`
    '''

    def K(x):
        N = x.shape[0]
        x = np.array(x, dtype=np.complex)
        
        x = np.hstack([[0], x, [0]])
        x = (np.roll(x,+1) - np.roll(x,-1))
        x *= I_diag(N+2, a, b) /2j
        rslt = x[1:-1]

        rslt[ 0] += x[-1]
        rslt[-1] += x[0]
        return rslt
    
    def Kinv(x):
        # Make sure type is coerced to complex, otherwise numpy ignores the complex parts
        # and reverts to reals.
        x = np.array(x.copy(), dtype=np.complex)
        N = x.shape[0]
        I = I_diag(N+2, a, b)
        x /= I[1:-1]

        if (np.isclose(I[0], I[N]) or 
            np.isclose(I[1], I[N+1]) or 
            np.isclose(I[0]*I[1], I[N]*I[N+1])):
            raise ValueError("Singular operator.")

        y = np.zeros(N, dtype=np.complex)
        # The computations below are essentially Schur's complement?
        E = np.sum(x[::2]); O = np.sum(x[1::2])
        if N % 2 == 0:
            y[0]  = O/(1-I[0]/I[N])
            y[-1] = E/(I[N+1]/I[1]-1)
        else:
            y[0]  = (I[1]/I[N+1]*E+O)/(1-I[1]*I[0]/I[N]/I[N+1])
            y[-1] = (I[N]/I[0]*E+O)/(I[N]*I[N+1]/I[0]/I[1]-1)
        
        x[0]  -= y[-1]*I[N+1]/I[1]
        x[-1] -= -y[0]*I[0]/I[N]

        # This should be the crux of the inverse
        x = np.hstack([[-y[0]], x , [y[-1]]])
        y[::2] = -np.cumsum(x[::2])[:-1]
        y[1::2] = np.cumsum(x[1::2][::-1])[:-1][::-1]

        y *= 2j

        return y
    
    return K, Kinv

In [39]:
K, Kinv = fourier_sin(0, 0.1)
x = np.ones(4)
y = K(x)
assert np.allclose(x, Kinv(y))

In [40]:
import dec.spectral as sp
from dec.helper import to_matrix

def Aa(x):
    f = np.hstack([ [-x[1]], x[:-2]-x[2:], [x[-2]] ])
    return f

print(to_matrix(Aa, 6))
#print(np.linalg.inv(to_matrix(Aa, 6)))

[[-0. -1. -0. -0. -0. -0.]
 [ 1.  0. -1.  0.  0.  0.]
 [ 0.  1.  0. -1.  0.  0.]
 [ 0.  0.  1.  0. -1.  0.]
 [ 0.  0.  0.  1.  0. -1.]
 [ 0.  0.  0.  0.  1.  0.]]


In [41]:
def fourier_sin_schur(a, b):

    def get_ABCD(n, m):
        
        N = n + m
        I = sp.I_diag(N+2, a, b) / 2j
        zeros = lambda n: np.zeros(n, dtype=np.complex)

        def A(x): 
            f = np.hstack([ [-x[1]], x[:-2]-x[2:], [x[-2]] ])
            f *= I[2:-2]
            return f
        
        def B(λ):
            f = zeros(n)
            f[0]  =  I[ 2]*λ[0]
            f[-1] = -I[N-1]*λ[1]
            return f

        def C(x):
            g  = zeros(m)
            g[0] = -I[1]*x[ 0]
            g[1] =  I[N]*x[-1]
            return g
        
        def D(λ):
            g = zeros(m)
            g[0] =  I[N+1]*λ[1]
            g[1] = -I[  0]*λ[0]
            return g
        
        return A, B, C, D

    def K(x):

        x = np.array(x, dtype=np.complex)
        
        x, λ = x[1:-1], x[[0, -1]]
        n = len(x)
        m = len(λ)

        A, B, C, D = get_ABCD(n, m)
        f = A(x) + B(λ)
        g = C(x) + D(λ)
        
        return np.hstack([ [g[0]], f, [g[1]] ])
    
    
    def Kinv(f):
        Minv = shur_lambda(A, B, C, D, n, m)
        return x
    
    return K, Kinv

K_, Kinv_ = fourier_sin(0, 1)
K, Kinv = fourier_sin_schur(0, 1)
X = np.arange(4)
Y = K_(X)
assert np.allclose(K(X), Y)
K(X).round(3), Y.round(3), Kinv_(Y).round(3)
K_(np.array([1,]))

array([ 0.45969769+0.j])

In [42]:
def I_diag_sy(N, a, b):
    from sympy import I, exp, Integer, symbols

    l = symbols(['I{}'.format(i) for i in range(N)])
#    l = []
#    for n in map(int, sp.freq(N)):
#         if n == 0:
#             l.append(b-a)
#         else:
#             n = Integer(n)
#             l.append( (exp(I*n*b) - exp(I*n*a))/(I*n) )/2/I
    return np.array(l)

def fourier_sin_schur_sym(a, b):
    from sympy import I, exp, Integer
        
    def K(x):
        N = x.shape[0]
        x = np.hstack([[0], x, [0]])

        x = (np.roll(x,+1) - np.roll(x,-1))
        x *= I_diag_sy(N+2, a, b)
        rslt = x[1:-1]

        rslt[ 0] += x[-1]
        rslt[-1] += x[0]
        return rslt

    def Kinv(x):
        pass
    
    return K, Kinv

import sympy as sy
a, b = sy.symbols('a b')
print(I_diag_sy(3, a, b))
K, Kinv = fourier_sin_schur_sym(a, b)

[I0 I1 I2]


In [43]:
X = np.array(sy.symbols('λ0 x0 x1 x2 x3 x4 x5 x6 x7 x8 λ1'))
K(X)

array([-I1*x0 + I12*λ1, I2*(-x1 + λ0), I3*(x0 - x2), I4*(x1 - x3),
       I5*(x2 - x4), I6*(x3 - x5), I7*(x4 - x6), I8*(x5 - x7),
       I9*(x6 - x8), I10*(x7 - λ1), -I0*λ0 + I11*x8], dtype=object)

In [44]:
# This is better because we have a lower triangular matrix.
X = np.array(sy.symbols('λ0 λ1 x0 x1 x2 x3 x4 x5 x6 x7 x8'))
K(X)

array([-I1*λ1 + I12*x8, I2*(-x0 + λ0), I3*(-x1 + λ1), I4*(x0 - x2),
       I5*(x1 - x3), I6*(x2 - x4), I7*(x3 - x5), I8*(x4 - x6),
       I9*(x5 - x7), I10*(x6 - x8), -I0*λ0 + I11*x7], dtype=object)

In [45]:
N = 1000
K, Kinv = fourier_sin(0, 0.1)
K_, Kinv_ = fourier_sin_schur(0, 0.1)
x = K(np.ones(N))
%timeit K(np.ones(N))
%timeit Kinv(x)
%timeit K_(x)

10000 loops, best of 3: 190 µs per loop
1000 loops, best of 3: 289 µs per loop
10000 loops, best of 3: 199 µs per loop
